In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect(':memory:')

In [4]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [5]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [11]:
pd.read_sql(
    '''
    SELECT * FROM OrderDetails
    ''',
    con,
)

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,11,12
1,2,10248,42,10
2,3,10248,72,5
3,4,10249,14,9
4,5,10249,51,40
...,...,...,...,...
513,514,10442,11,30
514,515,10442,54,80
515,516,10442,66,60
516,517,10443,11,6


In [9]:
pd.read_sql(
    '''
    SELECT * FROM Products
    WHERE ProductName like '%rab%'
    ''',
    con,
)

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,40,Boston Crab Meat,19,8,24 - 4 oz tins,18.4
1,61,Sirop d'érable,29,2,24 - 500 ml bottles,28.5


### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [12]:
pd.read_sql(
    '''
    SELECT CustomerName
          ,Country
          ,Address
    FROM Customers
    WHERE Country in ('Germany', 'France') OR City = 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [19]:
pd.read_sql(
    '''
    SELECT Country
          ,COUNT(*) as 'Customers_CNT'
    FROM Customers
    GROUP BY Country
    ORDER BY 2 DESC
    LIMIT 3
    ''',
    con,
)

,Country,Customers_CNT
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [60]:
pd.read_sql(
    '''
    SELECT s.ShipperName
          ,o.OrderDate
    FROM Orders o
        LEFT JOIN Shippers s
            ON o.ShipperID = s.ShipperID
    ORDER BY substr(o.OrderDate,7,4)||substr(o.OrderDate,4,2)||substr(o.OrderDate,1,2)
    LIMIT 1 OFFSET 9
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,16.07.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [80]:
pd.read_sql(
    '''
    SELECT ProductName
          ,Price
          ,Quantity
    FROM OrderDetails od
        LEFT JOIN Products p
            ON od.ProductID = p.ProductID
    WHERE OrderID = (
                    SELECT OrderID
                    FROM OrderDetails od
                        LEFT JOIN Products p
                            ON od.ProductID = p.ProductID
                    GROUP BY OrderID
                    ORDER BY SUM(Price * Quantity) DESC
                    LIMIT 1
                    )
                    
    ''',
    con,
)

,ProductName,Price,Quantity
0,Sir Rodney's Marmalade,81.0,12
1,Côte de Blaye,263.5,40
2,Camembert Pierrot,34.0,70
3,Mozzarella di Giovanni,34.8,42


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [91]:
pd.read_sql(
    '''
    SELECT p.ProductName
          ,od.OrderID
          ,od.Quantity
    FROM OrderDetails od
        LEFT JOIN Products p
            ON od.ProductID = p.ProductID
    WHERE od.ProductID = (
                        SELECT od.ProductID
                        FROM OrderDetails od
                            LEFT JOIN Products p
                                ON od.ProductID = p.ProductID
                        GROUP BY od.ProductID
                        ORDER BY SUM(Quantity) DESC
                        LIMIT 1
                        )
    ORDER BY 3 DESC
    ''',
    con,
)

,ProductName,OrderID,Quantity
0,Gorgonzola Telino,10359,70
1,Gorgonzola Telino,10390,60
2,Gorgonzola Telino,10342,56
3,Gorgonzola Telino,10376,42
4,Gorgonzola Telino,10272,40
5,Gorgonzola Telino,10393,32
6,Gorgonzola Telino,10356,30
7,Gorgonzola Telino,10374,30
8,Gorgonzola Telino,10335,25
9,Gorgonzola Telino,10253,20


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [106]:
pd.read_sql(
    '''
    SELECT s.SupplierName
          ,s.Country
          ,s.ContactName
          ,s.Phone
    
    FROM OrderDetails od
        LEFT JOIN Products p
            ON od.ProductID = p.ProductID
        LEFT JOIN Suppliers s
            ON s.SupplierID = p.SupplierID
    GROUP BY s.SupplierName
          ,s.Country
          ,s.ContactName
          ,s.Phone
    ORDER BY COUNT(DISTINCT od.OrderID) DESC
    LIMIT 5
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
1,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [136]:
pd.read_sql(
    '''
    SELECT cust.Country
          ,c.CategoryName
          ,SUM(p.Price * od.Quantity) as 'Sum'
    FROM OrderDetails od
        LEFT JOIN Products p
            ON p.ProductID = od.ProductID
        LEFT JOIN Categories c
            ON c.CategoryID = p.CategoryID
        LEFT JOIN Orders o
            ON o.OrderID = od.OrderID
        LEFT JOIN Customers cust
            ON cust.CustomerID = o.CustomerID
    WHERE cust.Country = 'Brazil'
    GROUP BY cust.Country
          ,c.CategoryName
    ORDER BY 3 DESC
    ''',
    con,
)

,Country,CategoryName,Sum
0,Brazil,Beverages,13690.00
1,Brazil,Confections,6307.30
2,Brazil,Produce,6205.45
3,Brazil,Dairy Products,5491.60
4,Brazil,Meat/Poultry,4780.85
5,Brazil,Condiments,2748.95
6,Brazil,Seafood,991.10


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [146]:
pd.read_sql(
    '''
    SELECT MAX(Sum) - MIN(Sum) as 'Diff'
    FROM (
            SELECT od.OrderID
                ,cust.Country
                ,SUM(p.Price * od.Quantity) as 'Sum'
            FROM OrderDetails od
                LEFT JOIN Products p
                    ON p.ProductID = od.ProductID
                LEFT JOIN Orders o
                    ON o.OrderID = od.OrderID
                LEFT JOIN Customers cust
                    ON cust.CustomerID = o.CustomerID
            WHERE cust.Country = 'USA'
            GROUP BY od.OrderID
                ,cust.Country
            ORDER BY 3 DESC
            ) t1
    ''',
    con,
)

,Diff
0,7638.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [155]:
pd.read_sql(
    '''
    SELECT COUNT(o.OrderID) as 'Orders_CNT'
          ,e.FirstName||' '||e.LastName as 'FIO'
    FROM Orders o
        LEFT JOIN Employees e
            ON e.EmployeeID = o.EmployeeID
    GROUP BY e.FirstName||' '||e.LastName, e.BirthDate
    ORDER BY substr(e.BirthDate,7,4)||substr(e.BirthDate,4,2)||substr(e.BirthDate,1,2) DESC
    LIMIT 3
    ''',
    con,
)

,Orders_CNT,FIO
0,6,Anne Dodsworth
1,29,Nancy Davolio
2,31,Janet Leverling


10. Сколько банок крабового мяса всего было заказано.

In [13]:
pd.read_sql(
    '''
    SELECT p.ProductName
          ,SUM(od.Quantity) as 'Total_Quantity'
    FROM OrderDetails od
        LEFT JOIN Products p
            ON od.ProductID = p.ProductID
    GROUP BY p.ProductName
    HAVING p.ProductName LIKE '%Crab%'
    ''',
    con,
)

,ProductName,Total_Quantity
0,Boston Crab Meat,256
